# OSM Oslo city bike data

The official [Oslo city bike app](https://oslobysykkel.no/en) doesn't work properly without internet connectivity. Many bysykkel station locations are still available using offline mobile apps, sush as the OSM based [maps.me](https://maps.me/en/home). However, not all of the stations are included in OSM. The purpose of this project is to look at the quality of the OSM data in this regard, and to learn how to better use python tools for processing and visualising geospatial data in a simple way

This document is written as a [Jupyter notebook](https://maps.me/en/home), you may be missing out on interactive elements if you are reading this on eg. GitHub.


# Authoritative data source 

Based on the map at https://oslobysykkel.no/en/map. The map is populated with json formatted data from https://oslobysykkel.no/api/internal/stations. 

In [78]:
import requests
import json
import geopandas as gpd
oslo_json = requests.get("https://oslobysykkel.no/api/internal/stations")
oslo_json = oslo_json.content.decode(oslo_json.apparent_encoding)
oslo_json = json.loads(oslo_json)
oslo_json['stations'][:3]

[{'center': {'latitude': 59.929838, 'longitude': 10.711223},
  'id': 178,
  'ready': True,
  'subtitle': 'langs Fridtjof Nansens vei',
  'title': 'Colosseum Kino'},
 {'center': {'latitude': 59.939228, 'longitude': 10.75917},
  'id': 158,
  'ready': True,
  'subtitle': 'rett over busstoppet',
  'title': 'Bentsebrugata'},
 {'center': {'latitude': 59.939238, 'longitude': 10.774279},
  'id': 159,
  'ready': True,
  'subtitle': 'Mellom Åsengata og Nordkappgata',
  'title': 'Hans Nielsen Hauges plass'}]

# OSM data

In [1]:
import requests
import json
osm_s = requests.get("https://gist.githubusercontent.com/jarmokivekas/b9d1cdfd006d3df26b871de28bebefe4/raw/450d1ab6f98de9307703583ea2655f29f688d09b/overpass.geojson")
osm_s = osm_s.content.decode(osm_s.apparent_encoding)
osm_s = json.loads(osm_s)

bysykkels = []
for feature in osm_s['features']:
    if feature['properties']['network'] == 'Bysykkel' :
        bysykkels.append(feature) 


In [67]:
import geopandas as gpd
import matplotlib.pyplot as plt
overpass_d = gpd.read_file("./overpass.geojson")
bysykkel_d = overpass_d[overpass_d['network'] == "Bysykkel"]

# print a few exmple points 
bysykkel_d.head(3)

,@id,addr:city,addr:country,addr:housenumber,addr:street,alt_name,amenity,capacity,description,geometry,name,network,note,operator,ref,source,wheelchair
0,node/408455943,None,None,None,None,None,bicycle_rental,12,33-Bygdøy v/Folkemuseet,POINT (10.6868353 59.9076592),None,Bysykkel,None,ClearChannel,33,survey,None
1,node/408457759,None,None,None,None,None,bicycle_rental,30,51-Havnegata S (Østbanehallen) v/Hotell Opera,POINT (10.7521008 59.910089),None,Bysykkel,None,ClearChannel,51,survey,None
2,node/408457760,None,None,None,None,None,bicycle_rental,30,50-Havnegata N (Østbanehallen),POINT (10.7517602 59.910162),None,Bysykkel,None,ClearChannel,50,survey,None


### Leflet.js visualization using folium

In [59]:
import folium

mapa = folium.Map(tiles='cartodbpositron')
mapa.fit_bounds([
    [bysykkel_d.bounds['miny'].min(),bysykkel_d.bounds['minx'].min()],
    [bysykkel_d.bounds['maxy'].max(),bysykkel_d.bounds['maxx'].max()]
])

bysykkel_geojson = bysykkel_d.to_json()
points = folium.features.GeoJson(bysykkel_geojson)
mapa.add_children(points)
mapa

One way of figuring out the bounds of a set of point:

In [82]:
bysykkel_d.bounds['miny'].min();
bysykkel_d.bounds['minx'].min();
bysykkel_d.bounds['maxy'].max();
bysykkel_d.bounds['maxx'].max();



pandas has a `DataFrame.mean()` for calculating the mean value of a data column. However, due to the way points are encoded as `shaplely.geometry.Point` I haven't figured out how to use `DataFrame.mean()` on location data.

In [81]:
x_cumulative_sum = 0
y_cumulative_sum = 0
for point in bysykkel_d['geometry']:
    x_cumulative_sum += point.x
    y_cumulative_sum += point.y

x_cumulative_sum / len(bysykkel_d['geometry']);
y_cumulative_sum / len(bysykkel_d['geometry']);